In [165]:
# == importar bibliotecas == 

from bs4 import BeautifulSoup as bs
import unicodedata as ucd
import requests
import pandas as pd
import numpy as np

In [266]:
# == settando variáveis == 

scrap = pd.DataFrame()
JORNAL = 'PSOL'
LABEL = 'esquerda' 
URL = "https://psol50.org.br/noticias/"
class_1 = 'entry-title td-module-title'
class_2 = 'td_block_wrap tdb_single_content tdi_75 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'

In [267]:
# == obter html == 

def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = bs(html, 'lxml')
    return soup

In [268]:
# == regra de restrição na seleção (remover trechos que nn fazem parte do artigo) == 

def check(text):
    restricoes = ['<a', '<href']
    if any(map(text.__contains__, restricoes)):
        return False
    return True

In [269]:
# == obter dataframe do texto == 

def return_df(url, text):
    return pd.DataFrame({
        'jornal': JORNAL,
        'url': url,
        'artigo': text,
        'label': LABEL
    }, index=[1])

In [270]:
# == lista de URLs ==

soup = get_soup(URL)
list_url = []

In [271]:
# == coletar os sites de artigos == 

for noticia in soup.find_all("h3", class_=class_1):
    for artigo in noticia.find_all("a", href=True):
        list_url.append(artigo.get('href'))

In [272]:
# == remover url duplicado == 

list_url = list(set(list_url))

In [273]:
# == obter conteúdo da notícia == 

for url in list_url:
    soup = get_soup(url)
    list_text = []
    for div in soup.find_all("div", class_=class_2):
        for p in div.find_all("p"):
            if check(str(p)):
                list_text.append(ucd.normalize("NFKD", p.text))
        scrap = pd.concat([
            scrap,
            return_df(url, " ".join(list_text))
        ])


In [274]:
# == salvar o scrapping == 

scrap.to_parquet('../dataset/scrapping/PSOL.parquet')

In [275]:
scrap.head(10)

,jornal,url,artigo,label
1,PSOL,https://psol50.org.br/luta-contra-a-extrema-di...,A segunda resolução aprovada no 8o Congresso...,esquerda
1,PSOL,https://psol50.org.br/psol-e-pt-acionam-stf-pa...,"O PSOL, ao lado do PT, protocolou na última s...",esquerda
1,PSOL,https://psol50.org.br/com-relatorio-de-tarcisi...,A Comissão de Constituição e Justiça da Ca...,esquerda
1,PSOL,https://psol50.org.br/psol-democratico-forte-e...,O 8o Congresso Nacional do PSOL aprovou uma re...,esquerda
1,PSOL,https://psol50.org.br/psol-aciona-ministerio-p...,"Os deputados federais do PSOL Erika Hilton, Pa...",esquerda
1,PSOL,https://psol50.org.br/nota-do-psol-pt-e-pcdob-...,Editorial do Estadão do dia 13 de outubro ata...,esquerda
1,PSOL,https://psol50.org.br/projeto-de-erika-hilton-...,A deputada federal do PSOL Erika Hilton aprese...,esquerda
1,PSOL,https://psol50.org.br/outra-cidade-e-possivel-...,"Nesta sexta-feira (20), a partir das 18h na se...",esquerda
1,PSOL,https://psol50.org.br/psol-reafirma-importanci...,O plenário do 8o Congresso Nacional do PSOL a...,esquerda
1,PSOL,https://psol50.org.br/projeto-de-celia-xakriab...,A deputada federal indígena Célia Xakriabá ...,esquerda
